# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [33]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [49]:
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df.columns = df.iloc[0]
df = df[1:]
tickers = df['Symbol']
tickers[:5]
problem_tickers = ['BRK-B','BF-B'] #discovered when trying to do api calls using those tickers
for p in problem_tickers:
    try:
        tickers.remove(p)
    except:
        continue

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
from secrets import IEX_CLOUD_TOKEN_SANDBOX

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock


In [7]:
all_data = []
for t in tickers:
    test_url = 'https://sandbox.iexapis.com/stable/stock/{}/quote/?token={}'.format(t, IEX_CLOUD_TOKEN_SANDBOX)
    try:
        data = requests.get(test_url).json()
    except:
        print(t)
        continue
    all_data.append(data)
print(all_data)

BRK-B
BF-B
[{'symbol': 'MMM', 'companyName': '3M Co.', 'primaryExchange': 'GI CE NA.O XHKO NTECYN,ERWCSK', 'calculationPrice': 'close', 'open': 177.31, 'openTime': 1641488185738, 'openSource': 'cfioflia', 'close': 176.55, 'closeTime': 1663552105628, 'closeSource': 'foiiaclf', 'high': 179.465, 'highTime': 1612158232748, 'highSource': 'mee  ypeatr1dud5il ecin', 'low': 176.96, 'lowTime': 1681220438243, 'lowSource': ' inm yadciedeeul1epr5 t', 'latestPrice': 177.87, 'latestSource': 'Close', 'latestTime': 'December 22, 2020', 'latestUpdate': 1621963010536, 'latestVolume': 1433476, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 175.25, 'delayedPriceTime': 1609108643103, 'oddLotDelayedPrice': 181.34, 'oddLotDelayedPriceTime': 1648221372375, 'extendedPrice': 181.82, 'extendedChange': -0.77, 'extendedChangePercent': -0.00456, 'extendedPriceTime': 1646291599659, 'previousClose': 176.25, 'previousVolume': 1724820, 'change': -0.9, 'changePercent': -0.0051

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [29]:
columns = ['Ticker', 'Stock Price', 'Market Cap', 'No. of Shares to Buy']
final_df = pd.DataFrame(columns = columns)
final_df

,Ticker,Stock Price,Market Cap,No. of Shares to Buy


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [31]:
%%time
for d in all_data:
    symbol, close, marketCap, shares = d['symbol'], d['close'], d['marketCap'], 'NA' 
    s = pd.Series([symbol, close, marketCap, shares], index=columns)
    final_df = final_df.append(s, ignore_index=True)
final_df.head()

Wall time: 1.28 s


,Ticker,Stock Price,Market Cap,No. of Shares to Buy
0,MMM,176.55,101228023940,NA
1,ABT,111.47,191940978787,NA
2,ABBV,106.9,187576838865,NA
3,ABMD,304.64,13830650768,NA
4,ACN,264.74,178217738655,NA


In [32]:
%%time
for idx, d in enumerate(all_data):
    symbol, close, marketCap, shares = d['symbol'], d['close'], d['marketCap'], 'NA' 
    s = [symbol, close, marketCap, shares]
    final_df.loc[idx] = s
final_df.head()

Wall time: 108 ms


,Ticker,Stock Price,Market Cap,No. of Shares to Buy
0,MMM,176.55,101228023940,NA
1,ABT,111.47,191940978787,NA
2,ABBV,106.9,187576838865,NA
3,ABMD,304.64,13830650768,NA
4,ACN,264.74,178217738655,NA


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [34]:
def chunks(lst, n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [92]:
%%time
symbol_groups = list(chunks(tickers, 100))
symbol_strings = []
for i in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns=columns)
idx = 0

for symbol_string in symbol_strings:
    batch_api_call = 'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={}&types=quote&token={}'.format(symbol_string, IEX_CLOUD_TOKEN_SANDBOX)
    data = requests.get(batch_api_call).json()
    for s in symbol_string.split(','):
        close, marketCap, shares = data[s]['quote']['close'], data[s]['quote']['marketCap'], 'NA'
        s_data = [s, close, marketCap, shares]
        final_dataframe.loc[idx] = s_data
        idx += 1
final_dataframe.dropna(subset=['Stock Price'], inplace=True)
final_dataframe.reset_index(inplace=True)
final_dataframe.drop(columns=['index'], inplace=True)
final_dataframe.head()

Wall time: 4.79 s


,Ticker,Stock Price,Market Cap,No. of Shares to Buy
0,MMM,175.54,102520386187,NA
1,ABT,111.78,194926305619,NA
2,ABBV,105.09,185928317511,NA
3,ABMD,311.01,14172913257,NA
4,ACN,268.6,173142009814,NA


In [45]:
data

{'MMM': {'quote': {'symbol': 'MMM',
   'companyName': '3M Co.',
   'primaryExchange': '  .HECOCG NENIW,ACKXORT SKENY',
   'calculationPrice': 'close',
   'open': 180.11,
   'openTime': 1688885187616,
   'openSource': 'iolfcaif',
   'close': 178.12,
   'closeTime': 1609296264591,
   'closeSource': 'fafilico',
   'high': 176.584,
   'highTime': 1659154595255,
   'highSource': 'nea51cdmd ye euteil irp',
   'low': 177.58,
   'lowTime': 1672960620191,
   'lowSource': 'ieadndi  ulee p5cmry1te',
   'latestPrice': 181.2,
   'latestSource': 'Close',
   'latestTime': 'December 24, 2020',
   'latestUpdate': 1687211448733,
   'latestVolume': 684865,
   'iexRealtimePrice': None,
   'iexRealtimeSize': None,
   'iexLastUpdated': None,
   'delayedPrice': 179.01,
   'delayedPriceTime': 1659511922462,
   'oddLotDelayedPrice': 174.52,
   'oddLotDelayedPriceTime': 1667790079838,
   'extendedPrice': 179.24,
   'extendedChange': 0,
   'extendedChangePercent': 0,
   'extendedPriceTime': 1654533969060,
   'pr

In [48]:
final_dataframe.shape

(503, 4)

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [61]:
def get_portfolio_value():
    portfolio_size = input('Enter portfolio value:')
    try:
        val = float(portfolio_size)
        return val
    except ValueError:
        print('Please enter an integer')
        get_portfolio_value()

portfolio_size = get_portfolio_value()

Enter portfolio value:1293012


In [93]:
final_dataframe.shape

(502, 4)

In [98]:
n, _ = final_dataframe.shape
position_size = portfolio_size/n
for i in range(n):
    final_dataframe.loc[i, 'No. of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

final_dataframe['Position Value'] = final_dataframe['Stock Price']*final_dataframe['No. of Shares to Buy'] 
final_dataframe.head()

,Ticker,Stock Price,Market Cap,No. of Shares to Buy,Position Value
0,MMM,175.54,102520386187,14,2457.56
1,ABT,111.78,194926305619,23,2570.94
2,ABBV,105.09,185928317511,24,2522.16
3,ABMD,311.01,14172913257,8,2488.08
4,ACN,268.6,173142009814,9,2417.4


In [100]:
buffer = portfolio_size - final_dataframe['Position Value'].sum()
buffer_percentage = buffer/portfolio_size
print(buffer_percentage)

0.033360974221429474


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [135]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [136]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border':1        
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border':1        
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border':1        
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [137]:
column_formats = {
    'A': ['Ticker',string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Cap',dollar_format],
    'D': ['No. of Shares to Buy',integer_format]    
}

for column, form in column_formats.items():
    writer.sheets['Recommended Trades'].set_column('{col}:{col}'.format(col=column), 18, form[1])
    writer.sheets['Recommended Trades'].write('{col}1'.format(col=column), form[0], string_format)
writer.save()